# setup

In [1]:
!pip install pytube
!pip install yt_dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 778.4 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 7.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import datetime
import pandas as pd
import pytube as pt
import yt_dlp
from tqdm.notebook import tqdm

path = "/content/drive/MyDrive/uraradi_transcript/"

# load playlist and transform to dataframe

In [11]:
playlist_url = "https://youtube.com/playlist?list=PLShwbdwZFm3r77Bwrr1quz2CpqJc6BZVL"
pl = pt.Playlist(url=playlist_url)

videos_columns = ["url", "title", "date", "length_s"]
videos_list = []

ydl_opts = {
    "ignore_no_formats_error": True
}

for url in pl.video_urls:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
    if "duration" in info.keys():
        title = info["title"]
        publish_date = str(info["release_date"][0:4]).zfill(4) + "-" + str(info["release_date"][4:6]).zfill(2) + "-" + str(info["release_date"][6:8]).zfill(2)
        length = info["duration"]
        videoinfo = [url, title, publish_date, length]
        videos_list.append(videoinfo)
        print("*"*50)
        print(videoinfo)
        print("*"*50)
df = pd.DataFrame(videos_list, columns=videos_columns)
df.sort_values(by="date", ascending=True).reset_index(drop=True)
df.head()

[youtube] Extracting URL: https://www.youtube.com/watch?v=YCvPUXbebzg
[youtube] YCvPUXbebzg: Downloading webpage
[youtube] YCvPUXbebzg: Downloading android player API JSON
**************************************************
['https://www.youtube.com/watch?v=YCvPUXbebzg', 'ラジオネームChatGPT｜#74.5 裏ラジオウルナイト｜大浦るかこ // あにまーれ', '2023-03-10', 2766]
**************************************************
[youtube] Extracting URL: https://www.youtube.com/watch?v=0-WWGaTfqW4
[youtube] 0-WWGaTfqW4: Downloading webpage
[youtube] 0-WWGaTfqW4: Downloading android player API JSON


[youtube] Extracting URL: https://www.youtube.com/watch?v=o61S1MnpG9c
[youtube] o61S1MnpG9c: Downloading webpage
[youtube] o61S1MnpG9c: Downloading android player API JSON
**************************************************
['https://www.youtube.com/watch?v=o61S1MnpG9c', '熱が冷めるとき｜#74 裏ラジオウルナイト｜大浦るかこ // あにまーれ', '2023-03-03', 6722]
**************************************************
[youtube] Extracting URL: https://www.youtube.com/watch?v=DjD9aKYtVz0
[youtube] DjD9aKYtVz0: Downloading webpage
[youtube] DjD9aKYtVz0: Downloading android player API JSON
**************************************************
['https://www.youtube.com/watch?v=DjD9aKYtVz0', '【裏ラジ#73】母校の様子が変 / 裏ラジオウルナイト【大浦るかこ / あにまーれ】', '2023-02-24', 6272]
**************************************************
[youtube] Extracting URL: https://www.youtube.com/watch?v=MpPw4htq3ZE
[youtube] MpPw4htq3ZE: Downloading webpage
[youtube] MpPw4htq3ZE: Downloading android player API JSON
**************************************************
['https

,url,title,date,length_s
0,https://www.youtube.com/watch?v=YCvPUXbebzg,ラジオネームChatGPT｜#74.5 裏ラジオウルナイト｜大浦るかこ // あにまーれ,2023-03-10,2766
1,https://www.youtube.com/watch?v=o61S1MnpG9c,熱が冷めるとき｜#74 裏ラジオウルナイト｜大浦るかこ // あにまーれ,2023-03-03,6722
2,https://www.youtube.com/watch?v=DjD9aKYtVz0,【裏ラジ#73】母校の様子が変 / 裏ラジオウルナイト【大浦るかこ / あにまーれ】,2023-02-24,6272
3,https://www.youtube.com/watch?v=MpPw4htq3ZE,【裏ラジ#72】葬式で流す曲 / 裏ラジオウルナイト【龍ヶ崎リン・大浦るかこ / あにまーれ】,2023-02-17,8647
4,https://www.youtube.com/watch?v=5M9wGdW7ko0,【裏ラジ#71】珈琲vs紅茶 / 裏ラジオウルナイト【ハヤシ・大浦るかこ / あにまーれ】,2023-02-10,6681


# export as csv

In [12]:
filename = path + "playlist_" + pl.title + ".csv"

df.to_csv(filename, index=False)